In [334]:
import pandas as pd 
import numpy as np
import re
import os


In [335]:
pd.options.display.max_rows = 50

# Importing the data file
sharksDf = pd.read_csv("data/attacks.csv",encoding = "ISO-8859-1") 

#El uso del encoding viene a raiz de un error que tenía si no lo ponía: 
# https://stackoverflow.com/questions/18171739/unicodedecodeerror-when-reading-csv-file-in-pandas-with-python

In [336]:
sharksDf.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


- Remove the columns that we don't care about
- Remove duplicate columns
- Remove duplicate rows(Same order number? case number?)
- Check if Case Number.1 and Case Number.2 are ever different, if not remove column)
- Sort between fatal and non fatal attacks - create a new column
- Replace unvalid responses with "null"

In [337]:
sharksDf.shape

(25723, 24)

## We remove the rows and columns that we don't need for a statistical analysis

In [338]:
sharksDf = sharksDf.drop(columns=["Case Number","Name", "Injury", "Investigator or Source", "pdf", "href formula","href","original order","Case Number.1","Case Number.2"])
sharksDf = sharksDf.drop('Unnamed: 22', 1)
sharksDf = sharksDf.drop('Unnamed: 23', 1)

In [339]:
#Removing null rows and duplicates
sharksDf = sharksDf.dropna(how='all', axis=0)
sharksDf = sharksDf.drop_duplicates()




In [340]:
null_cols = sharksDf.isnull().sum()
null_cols[null_cols > 0]

Year              2
Type              4
Country          49
Area            452
Location        536
Activity        542
Sex             565
Age            2815
Fatal (Y/N)     539
Time           3339
Species        2827
dtype: int64

## Now I proceed to cleaning the different columns

   ### Cleaning "Type" column

In [341]:
df = sharksDf["Type"].value_counts()
df.head(30)

# Type should be either "provoked" or "unprovoked"

Unprovoked      4581
Provoked         572
Invalid          547
Sea Disaster     238
Boating          203
Boat             137
Questionable       2
Boatomg            1
Name: Type, dtype: int64

It seems like some mistake was made when inputing 'Boating' or  'Sea Disaster'; those inputs correspond more to the 
"Activity" column. To have a cleaner dataframe, I've decided to only keep 3 categories: Provoked, Unprovoked, and Uncertain

In [342]:
# Replacing all incorrect entries with "Uncertain"
sharksDf = sharksDf.replace({'Type': {"Invalid": "Uncertain", "Sea Disaster": "Uncertain","Boating": "Uncertain", "Boat": "Uncertain", "Questionable": "Uncertain", "Boatomg": "Uncertain"}})

# Replacing null values with the type "Uncertain"
sharksDf[sharksDf['Type'].isnull()]=sharksDf[sharksDf['Type'].isnull()].fillna('Uncertain')
sharksDf.Type.isnull().sum()

0

In [343]:
sharksDf["Type"].value_counts()

Unprovoked    4581
Uncertain     1132
Provoked       572
Name: Type, dtype: int64

In [344]:
null_cols = sharksDf.isnull().sum()
null_cols[null_cols >= 0]


Date              0
Year              2
Type              0
Country          49
Area            450
Location        535
Activity        542
Sex             565
Age            2812
Fatal (Y/N)     538
Time           3336
Species        2823
dtype: int64

Now, the 'Type' column is nice and clean

In [345]:
sharksDf["Year"].value_counts()

2015.0    143
2017.0    136
2016.0    130
2011.0    128
2014.0    127
         ... 
1801.0      1
1638.0      1
1834.0      1
1723.0      1
1786.0      1
Name: Year, Length: 249, dtype: int64

  ### Cleaning "Year" column

In [346]:
def checkDate(dateString):
    dateString=str(dateString)
    if re.findall(r'[0-9]{4}',dateString):
        return re.findall(r'[0-9]{4}',dateString)[0]
    else:
        return "unknown"


#2-Storage of the guessed year on a temporary support column 'DataTent'                                                                                                                            
sharksDf['Year']=sharksDf['Date'].apply(checkDate) 

In [347]:
null_cols = sharksDf.isnull().sum()
null_cols[null_cols >= 0]

Date              0
Year              0
Type              0
Country          49
Area            450
Location        535
Activity        542
Sex             565
Age            2812
Fatal (Y/N)     538
Time           3336
Species        2823
dtype: int64

  ### Cleaning "Country" column

In [348]:
sharksDf['Country'].value_counts()
sharksDf['Country'].isnull().sum()
sharksDf=sharksDf.fillna('Unknown')
sharksDf

,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Fatal (Y/N),Time,Species
0,25-Jun-2018,2018,Uncertain,USA,California,"Oceanside, San Diego County",Paddling,F,57,N,18h00,White shark
1,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,F,11,N,14h00 -15h00,Unknown
2,09-Jun-2018,2018,Uncertain,USA,Hawaii,"Habush, Oahu",Surfing,M,48,N,07h45,Unknown
3,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,M,Unknown,N,Unknown,2 m shark
4,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,M,Unknown,N,Unknown,"Tiger shark, 3m"
...,...,...,...,...,...,...,...,...,...,...,...,...
6297,Before 1903,1903,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,M,Unknown,Y,Unknown,Unknown
6298,Before 1903,1903,Unprovoked,AUSTRALIA,Western Australia,Unknown,Pearl diving,M,Unknown,Y,Unknown,Unknown
6299,1900-1905,1900,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,M,Unknown,Y,Unknown,Unknown
6300,1883-1889,1883,Unprovoked,PANAMA,Unknown,"Panama Bay 8ºN, 79ºW",Unknown,M,Unknown,Y,Unknown,Unknown


In [349]:
null_cols = sharksDf.isnull().sum()
null_cols[null_cols >= 0]


Date           0
Year           0
Type           0
Country        0
Area           0
Location       0
Activity       0
Sex            0
Age            0
Fatal (Y/N)    0
Time           0
Species        0
dtype: int64

  ### Cleaning "Species" column

In [350]:
sharksDf["Species "].value_counts()

Unknown                                                    2824
White shark                                                 163
Shark involvement prior to death was not confirmed          105
Invalid                                                     102
Shark involvement not confirmed                              88
                                                           ... 
0.9 m [3'] shark,  probably a blacktip or spinner shark       1
White shark, 12' to 15' female                                1
7' to 8' bull shark                                           1
1.8 m to 2.1 m [6' to 7'] spinner or blacktip shark           1
Said to involve  9' blue shark                                1
Name: Species , Length: 1550, dtype: int64

In [351]:
def sharkType(string):
    string = string.upper()
    if (len(re.findall(r'[A-Z]+\sSHARK', string))) > 0:
        return "".join(re.findall(r'[A-Z]+\sSHARK', string))
    else:
        return "Unknown"


sharksDf["Species "] = sharksDf["Species "].apply(sharkType)

sharksDf['Species ']=sharksDf['Species '].replace('M SHARK','Unknown')


In [352]:
sharksDf

,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Fatal (Y/N),Time,Species
0,25-Jun-2018,2018,Uncertain,USA,California,"Oceanside, San Diego County",Paddling,F,57,N,18h00,WHITE SHARK
1,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,F,11,N,14h00 -15h00,Unknown
2,09-Jun-2018,2018,Uncertain,USA,Hawaii,"Habush, Oahu",Surfing,M,48,N,07h45,Unknown
3,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,M,Unknown,N,Unknown,Unknown
4,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,M,Unknown,N,Unknown,TIGER SHARK
...,...,...,...,...,...,...,...,...,...,...,...,...
6297,Before 1903,1903,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,M,Unknown,Y,Unknown,Unknown
6298,Before 1903,1903,Unprovoked,AUSTRALIA,Western Australia,Unknown,Pearl diving,M,Unknown,Y,Unknown,Unknown
6299,1900-1905,1900,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,M,Unknown,Y,Unknown,Unknown
6300,1883-1889,1883,Unprovoked,PANAMA,Unknown,"Panama Bay 8ºN, 79ºW",Unknown,M,Unknown,Y,Unknown,Unknown


In [353]:
sharksDf["Species "].value_counts()

Unknown                                          4233
WHITE SHARK                                       622
TIGER SHARK                                       258
BULL SHARK                                        170
NURSE SHARK                                        95
                                                 ... 
WHITE SHARKTHRESHER SHARK                           1
CARCHARHINID SHARK                                  1
M SHARKREEF SHARK                                   1
WHITETIP SHARKBLUE SHARKTIGER SHARKBULL SHARK       1
WHITE SHARKSEVENGILL SHARK                          1
Name: Species , Length: 132, dtype: int64

  ### Cleaning "Activity" column

In [354]:
sharksDf["Activity"].value_counts()

Surfing                                                                                                                            971
Swimming                                                                                                                           865
Unknown                                                                                                                            545
Fishing                                                                                                                            430
Spearfishing                                                                                                                       331
                                                                                                                                  ... 
Attempting to retreive a dinghy                                                                                                      1
Treading for clams                                     

In [355]:
def activity(string):
    string = string.upper()
    if (len(re.findall(r'SURF|BOARD', string))) > 0:
        return "SURFING/WATERSPORTS"
    elif (len(re.findall(r'SWIM|DIVE|BATH|DIVING|SNORKELING|WADING', string))) > 0:
        return "SWIMMING/DIVING"
    elif (len(re.findall(r'FISH', string))) > 0:
        return "FISHING"
    else:
        return string
    
sharksDf["Activity"] = sharksDf["Activity"].apply(activity)

In [356]:
sharksDf["Activity"].value_counts()

SWIMMING/DIVING                                                                                                                                                                                           2100
SURFING/WATERSPORTS                                                                                                                                                                                       1561
FISHING                                                                                                                                                                                                   1095
UNKNOWN                                                                                                                                                                                                    545
STANDING                                                                                                                                                                    

In [357]:
os.getcwd()
sharksDf.to_csv("/Users/umberto/Desktop/Datamad/JAWS_Analyticssharks_clean.csv")

'/Users/umberto/Desktop/Datamad/JAWS_Analytics'